# Serverside Evaluation and Batch Trace Ingestion with Snowflake

This notebook walks through the complete TruLens + Snowflake experience.

This setup offers two advantages compared to other ways of use:
- Batch ingestion of records (traces) to Snowflake offers a faster ingestion experience
- Compuation of Evaluations on the Snowflake warehouse (serverside) removes the computation from the client

## Step 1: Connect to Snowflake for Logging Traces and Evaluations

Notice we're setting the `init_server_side` parameter to `True`. This will trigger uploading the tasks, streams and stored procedures to your Snowflake account needed to compute evaluations in the warehouse.

In [ ]:
from trulens.connectors.snowflake import SnowflakeConnector
from trulens.core.schema.app import RecordIngestMode
from trulens.core.session import TruSession

connection_params = {
    "account": "...",
    "user": "...",
    "password": "...",
    "database": "...",
    "schema": "...",
    "warehouse": "...",
    "role": "...",
    "init_server_side": True,  # Set to True to enable server side feedback functions
}

connector = SnowflakeConnector(**connection_params)
session = TruSession(connector=connector)

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(session=session)

## Connect to Cortex Search

In [ ]:
from typing import List

from snowflake.core import Root
from snowflake.snowpark import Session

snowpark_session = Session.builder.configs(connection_params).create()


class CortexSearchRetriever:
    def __init__(self, session: Session, limit_to_retrieve: int = 4):
        self._session = session
        self._limit_to_retrieve = limit_to_retrieve

    def retrieve(self, query: str) -> List[str]:
        root = Root(self._session)
        cortex_search_service = (
            root.databases["JREINI_DB"]
            .schemas["TRULENS_DEMO_SCHEMA"]
            .cortex_search_services["TRULENS_DEMO_CORTEX_SEARCH_SERVICE"]
        )
        resp = cortex_search_service.search(
            query=query,
            columns=["doc_text"],
            limit=self._limit_to_retrieve,
        )

        if resp.results:
            return [curr["doc_text"] for curr in resp.results]
        else:
            return []

## Step 2: Instrument an existing app

In [ ]:
from snowflake.cortex import Complete
from trulens.apps.custom import instrument


class RAG_from_scratch:
    def __init__(self):
        self.retriever = CortexSearchRetriever(
            session=snowpark_session, limit_to_retrieve=4
        )

    @instrument
    def retrieve_context(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        return self.retriever.retrieve(query)

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        prompt = f"""
          You are an expert assistant extracting information from context provided.
          Answer the question based on the context. Be concise and do not hallucinate.
          If you don´t have the information just say so.
          Context: {context_str}
          Question:
          {query}
          Answer:
        """
        return Complete("mistral-large", prompt)

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve_context(query)
        return self.generate_completion(query, context_str)


rag = RAG_from_scratch()

## Step 3: Define evaluations to run on Snowflake

By simply using the `SnowflakeFeedback` class isntead of `Feedback`, we specify that these feedback functions will run in Snowflake.

In [ ]:
import numpy as np
from trulens.core import Select
from trulens.core.feedback.feedback import SnowflakeFeedback
from trulens.providers.cortex import Cortex

provider = Cortex(
    connection_params,
    model_engine="mistral-large2",
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    SnowflakeFeedback(
        provider.relevance_with_cot_reasons, name="Answer Relevance"
    )
    .on_input()
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    SnowflakeFeedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(Select.RecordCalls.retrieve_context.rets)
    .aggregate(np.mean)
)

f_groundedness = (
    SnowflakeFeedback(
        provider.groundedness_measure_with_cot_reasons,
        name="Groundedness",
        use_sent_tokenize=False,
    )
    .on_input()
    .on(Select.RecordCalls.retrieve_context.rets.collect())
)

## Step 4: Register the app with TruLens

Here we add the new record ingest mode parameter set to buffered. This means that the records (traces) will be sent to Snowflake in batches.

In [ ]:
from trulens.apps.custom import TruCustomApp

tru_rag = TruCustomApp(
    rag,
    app_name="RAG",
    app_version="base",
    feedbacks=[
        f_answer_relevance,
        f_context_relevance,
        f_groundedness,
    ],
    record_ingest_mode=RecordIngestMode.BUFFERED,
)

## Set test set

In [ ]:
queries = [
    "How do I deploy streamlit in the cloud?",
    "What is the best way to deploy a streamlit app?",
    "How do I use streamlit buttons?",
    "How do I change the color of the background of a streamlit app?",
    "How do I add a logo to a streamlit app?",
    "How do I deploy streamlit in the cloud?",
    "What is the best way to deploy a streamlit app?",
    "How do I use streamlit buttons?",
    "How do I change the color of the background of a streamlit app?",
    "How do I add a logo to a streamlit app?",
    "How do I deploy streamlit in the cloud?",
    "What is the best way to deploy a streamlit app?",
    "How do I use streamlit buttons?",
    "How do I change the color of the background of a streamlit app?",
    "How do I add a logo to a streamlit app?",
    "How do I deploy streamlit in the cloud?",
    "What is the best way to deploy a streamlit app?",
    "How do I use streamlit buttons?",
    "How do I change the color of the background of a streamlit app?",
    "How do I add a logo to a streamlit app?",
]

## Step 5: Record application traces

In [ ]:
for query in queries:
    with tru_rag as recording:
        resp = rag.query(query)

## Optional: Improve the app

In [ ]:
from trulens.core.feedback.feedback import Feedback
from trulens.core.guardrails.base import context_filter

# note: feedback function used for guardrail must only return a score, not also reasons
f_context_relevance_score = Feedback(
    provider.context_relevance, name="Context Relevance"
)


class filtered_RAG_from_scratch(RAG_from_scratch):
    @instrument
    @context_filter(f_context_relevance_score, 0.75, keyword_for_prompt="query")
    def retrieve_context(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        return self.retriever.retrieve(query)


filtered_rag = filtered_RAG_from_scratch()

In [ ]:
from trulens.apps.custom import TruCustomApp

tru_filtered_rag = TruCustomApp(
    filtered_rag,
    app_name="RAG",
    app_version="filtered context",
    feedbacks=[
        f_answer_relevance,
        f_context_relevance,
    ],
    record_ingest_mode=RecordIngestMode.BUFFERED,
)

In [ ]:
for query in queries:
    with tru_filtered_rag as recording:
        resp = filtered_rag.query(query)